In [10]:
import pandas as pd
data = pd.read_json("../input/Restor_Eco/restor_eco.json")

In [11]:
data.head(4)

,id,name,description,polygon,siteType,siteVisibility,surfaceAreaKm2,countryCode,interventionStartYear,stage,...,landTenureUrl,photoIds,customFields,customLinkUrl,customLinkText,donateUrl,analysis,isFeatured,createdAt,updatedAt
0,3eec6174-21ce-4cb0-9180-82c1ccbef0bb,[Mabale - Holistic land management site SFA],{},"{'type': ['MultiPolygon'], 'coordinates': [[[[...",[LANDSCAPE],[PUBLIC],[104.9347],[ZW],[2021],[ONGOING],...,{},[],[],{},{},{},{},[False],[2023-02-23T12:18:27.313Z],[2023-02-23T12:18:27.313Z]
1,6bbc2d6c-ce35-48b2-9fd4-af74030f7fda,[Lukango Tree Conservancy Limited (LuTreeCo)],[Lukango Tree Conservancy Limited (LuTreeCo) i...,"{'type': ['MultiPolygon'], 'coordinates': [[[[...",[RESTORATION],[PUBLIC],[0.3275],[UG],[2017],[ONGOING],...,{},"[1bfa2bcd-6505-405d-abf7-6bbc8113fde6, dc0ff07...",[],{},{},{},{},[False],[2023-04-11T08:54:50.837Z],[2023-04-11T14:01:54.154Z]
2,bccf3ff8-cc4e-47f2-9435-aaaa5ff9a89c,[Lilongwe (Not entire City)],[This is the city of Lilongwe which we have do...,"{'type': ['MultiPolygon'], 'coordinates': [[[[...",[RESTORATION],[PUBLIC],[362.3797],[MW],[2022],[ONGOING],...,{},[],[],{},{},{},{},[False],[2023-02-27T14:42:54.442Z],[2023-02-27T14:48:17.999Z]
3,c8ad81c0-6bec-43fe-a784-274b0cb54ec8,[Upemba National Park],{},"{'type': ['MultiPolygon'], 'coordinates': [[[[...",[CONSERVATION],[PUBLIC],[10006.1403],[CD],{},[PLANNING],...,{},[],[],{},{},{},{},[False],[2023-04-12T22:30:06.907Z],[2023-04-13T00:23:01.425Z]


In [12]:
data["website"]

0                                      {}
1                  [https://lutreeco.com]
2                 [http://www.casamw.org]
3                                      {}
4                                      {}
                      ...                
1535                                   {}
1536                                   {}
1537                                   {}
1538    [https://www.reforestaction.com/]
1539        [https://www.tonywild.co.ke/]
Name: website, Length: 1540, dtype: object

In [13]:
data.columns

Index(['id', 'name', 'description', 'polygon', 'siteType', 'siteVisibility',
       'surfaceAreaKm2', 'countryCode', 'interventionStartYear', 'stage',
       'preInterventionLandUse', 'interventionType',
       'postInterventionLandCover', 'siteManagement', 'siteOwnership', 'goals',
       'supportSought', 'website', 'contactEmail', 'landTenureUrl', 'photoIds',
       'customFields', 'customLinkUrl', 'customLinkText', 'donateUrl',
       'analysis', 'isFeatured', 'createdAt', 'updatedAt'],
      dtype='object')

In [14]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'id': 'project_id_reported',
    #'countPlanted': 'trees_planted_reported',
    'description': 'description_reported',
    'interventionStartYear': 'planting_date_reported',
    'polygon': 'geometry_reported',
    'website': 'url',
    'surfaceAreaKm2': 'site_sqkm',
     'countryCode': 'country'
    #'survivalRate': 'survival_rate_reported',
   
}


data.rename(columns=columns_rename_mapping, inplace=True)

In [15]:
columns_to_keep = [
     'project_id_reported',
    #'countPlanted': 'trees_planted_reported',
    'description_reported',
     'planting_date_reported',
     'geometry_reported',
     'url',
     'site_sqkm',
     'country'
]
# Filtering only the colunms we are interested  in as the data has 64 columns some of which are not useful in our analysis
data=data[columns_to_keep]


data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1540 entries, 0 to 1539
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   project_id_reported     1540 non-null   object
 1   description_reported    1540 non-null   object
 2   planting_date_reported  1540 non-null   object
 3   geometry_reported       1540 non-null   object
 4   url                     1540 non-null   object
 5   site_sqkm               1540 non-null   object
 6   country                 1540 non-null   object
dtypes: object(7)
memory usage: 96.2+ KB


In [16]:
data['site_id_reported'] = ['restor_site_{}'.format(i) for i in range(1, len(data) + 1)]
data = data.assign(species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1540 entries, 0 to 1539
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   project_id_reported       1540 non-null   object
 1   description_reported      1540 non-null   object
 2   planting_date_reported    1540 non-null   object
 3   geometry_reported         1540 non-null   object
 4   url                       1540 non-null   object
 5   site_sqkm                 1540 non-null   object
 6   country                   1540 non-null   object
 7   site_id_reported          1540 non-null   object
 8   species_count_reported    0 non-null      object
 9   species_planted_reported  0 non-null      object
 10  survival_rate_reported    0 non-null      object
 11  trees_planted_reported    0 non-null      object
dtypes: object(12)
memory usage: 156.4+ KB


In [17]:
#Saving The  data to csv
data.to_csv('../input/Restor_Eco/restor_eco.csv', index=False)